In [72]:
# from pymongo import MongoClient
# client = MongoClient("")
from pymongo import MongoClient, GEOSPHERE
import pandas as pd
import time
from collections import Counter

In [8]:
client = MongoClient("localhost:27017")

In [9]:
db = client["Ironhack"]

In [10]:
#geting the dataset
c=db.get_collection("companies")

In [11]:
# filter for the design companies
#only 4
filter_design = {"category_code":"design"}
projection_design = {"_id":0, "name":1, "offices.city":1}
list(c.find(filter_design, projection_design))

[{'name': '99designs',
  'offices': [{'city': 'San Francisco'},
   {'city': 'Collingwood'},
   {'city': 'Berlin'}]},
 {'name': 'Graticle', 'offices': [{'city': 'Ellensburg'}]},
 {'name': 'Kickstarter', 'offices': [{'city': 'Brooklyn'}]},
 {'name': 'Moonfruit', 'offices': [{'city': 'London'}]}]

In [13]:
#checking for the industry to define tech industry
list(c.distinct("category_code"))[:5]

[None, 'advertising', 'analytics', 'automotive', 'biotech']

In [14]:
#list with tech companies
tech = ['biotech','cleantech','ecommerce','games_video','nanotech','software','web']

In [15]:
#check list of latest founded year
list(c.distinct("founded_year"))[-4:]

[2010, 2011, 2012, 2013]

In [16]:
c.find_one()["funding_rounds"][0]['raised_amount']

5250000

In [17]:
# filter tech companies for startup criteria
filter_tech = {"category_code":{"$in":tech}}
filter_year = {"founded_year":{"$gte":2000}}
filter_funding = {"funding_rounds.raised_amount":{"$gte":1000000}}
filter_employees={"number_of_employees":{"$lte":100}}

In [18]:
#list of filters with all filters
all_filters=[ filter_tech, filter_year, filter_funding, filter_employees ]

In [63]:
#list with all filters applied to 'c'
projection_3 = {"_id":0,"offices.city":1}
x=list(c.find({"$and": all_filters}, projection_3))
x[:10]

[{'offices': [{'city': 'Seattle'}, {'city': 'New York'}]},
 {'offices': [{'city': 'West Hollywood'}]},
 {'offices': [{'city': 'New York'}]},
 {'offices': [{'city': 'Sunnyvale'}]},
 {'offices': [{'city': 'Culver City'}]},
 {'offices': [{'city': 'San Francisco'}]},
 {'offices': [{'city': 'New York'}]},
 {'offices': [{'city': 'Pleasanton'}]},
 {'offices': [{'city': 'Luxembourg City'},
   {'city': 'Palo Alto'},
   {'city': 'London'}]},
 {'offices': [{'city': 'San Bruno'}]}]

In [36]:
# Checking an individual element to design the for loop
x[0]['offices'][0]['city']

'Seattle'

In [52]:
# List of cities from offices from companies
cities=[j['city'] for i in x for j in i['offices'] ]
cities

['Seattle',
 'New York',
 'West Hollywood',
 'New York',
 'Sunnyvale',
 'Culver City',
 'San Francisco',
 'New York',
 'Pleasanton',
 'Luxembourg City',
 'Palo Alto',
 'London',
 'San Bruno',
 'NYC',
 'New York',
 'Encino',
 'San Francisco',
 'Amsterdam',
 'Redwood City',
 'San Francisco',
 'Paris',
 'San Francisco',
 'San Francisco',
 'Seattle',
 'Palo Alto',
 'Seattle',
 'San Francisco',
 'Charlottesville',
 'San Mateo',
 'San Diego',
 'Los Angeles',
 'San Francisco',
 'San Francisco',
 'San Francisco',
 'Montecito',
 'San Mateo',
 'San Francisco',
 'Huntsville',
 'San Mateo',
 'Seattle',
 'Seattle',
 None,
 'Menlo Park',
 None,
 'Mountain View',
 'Redmond',
 'San Francisco',
 'Cape Town',
 'San Francisco',
 'San Francisco',
 'San Francisco',
 'Palo Alto',
 'Durham',
 'Palo Alto',
 'Santa Monica',
 'Kuala Lumpur',
 'New York',
 'Boston',
 'New York',
 'Austin',
 'San Francisco',
 'New York',
 'San Mateo',
 'Los Angeles',
 'Palo Alto',
 'Frankfurt am Main',
 'Boulder',
 'Portland',
 '

In [62]:
#Top 10 count of city mentions.
counts = pd.Series(cities).value_counts()
print(counts[:10])

San Francisco    61
New York         57
London           31
Seattle          23
San Mateo        20
Palo Alto        16
Paris            15
Cambridge        12
Los Angeles      12
Sunnyvale        10
dtype: int64


In [ ]:
# Best cities to work as a Designer
# https://www.designrush.com/agency/graphic-design/trends/top-countries-and-cities-for-design-jobs#:~:text=New%20York%20City%20is%20an,of%20the%20best%20design%20talent
'''
Berlin
Toronto
Montreal
    New York
Boston
    San Francisco
    London
'''

In [ ]:
# Cross reference both and we got our 3 cities: San Francisco, New York & London.

In [67]:
# London Tech startup cluster
![East London]()

zsh: parse error near `()'


In [68]:
# New York Tech startup cluster
![Lower & Mid Manhattan]()

zsh: parse error near `()'


In [81]:
nyneigh = db.get_collection("nyneigh")

In [82]:
db.nyneigh.create_index([("location", GEOSPHERE)])

'location_2dsphere'

In [83]:
nyneigh.find_one()

{'_id': ObjectId('55cb9c666c522cafdb053a1a'),
 'geometry': {'coordinates': [[[-73.94193078816193, 40.70072523469547],
    [-73.9443878859649, 40.70042452378256],
    [-73.94424286147482, 40.69969927964773],
    [-73.94409591260093, 40.69897295461309],
    [-73.94394947271304, 40.69822127983908],
    [-73.94391750192877, 40.69805620211356],
    [-73.94380383211836, 40.697469265449826],
    [-73.94378455587042, 40.6973697290538],
    [-73.94374306706803, 40.69715549995503],
    [-73.9437245356891, 40.697059812179496],
    [-73.94368427322361, 40.696851909818065],
    [-73.9436842703752, 40.69685189440415],
    [-73.94363806934868, 40.69661331854307],
    [-73.94362121369004, 40.696526279661654],
    [-73.9435563415296, 40.69619128295102],
    [-73.94354024149403, 40.6961081421151],
    [-73.94352527471477, 40.69603085523812],
    [-73.94338802084431, 40.69528899051899],
    [-73.943242490861, 40.694557485733355],
    [-73.94312826743185, 40.693967038330925],
    [-73.94311427813774, 40.6

In [ ]:
with open ("distritos.geojson") as f:
    districts = json.load(f)
	
with open ("just_districts.json", "w") as file:
	json.dump(districts, file)